<br><br><b><font size=10> CONTINUOUS CONTROL</font></b>
#### <i>...implementation for Udacity Deep Reinforcement Learning 
<hr><hr><hr>

## Initialize Imports for the notebook
This Notebook uses code from separate python files where most of the implementation is handled

In [1]:
import environment as E
from buffers import ReplayBuffer, nStepBuffer
from agent import D4PG_Agent

#from get_args import get_args

import os.path
import time
import re
import sys
import importlib
import copy

import torch
import matplotlib.pyplot as plt
import numpy as np
from unityagents import UnityEnvironment
from collections import deque
import torchvision.transforms as T
import multiprocessing as multi
multi.cpu_count()

16

## Manually declare an ARGS class
<i> For testing code in the notebook, to take the place of argparser in the command line.

In [2]:
class Args:
    def __init__(self):
        self.train = True
        self.nographics = False
        self.num_eps = 10
        self.rollout = 5
        self.batchsize = 64
        self.pretrain = 1000
        
args = Args()

<b>Confirm that the args are all set the way we want them.

In [3]:
for arg in vars(args):
    if arg == "sep": continue
    print("{}: {}".format(arg.upper(), getattr(args, arg)))

TRAIN: True
NOGRAPHICS: False
NUM_EPS: 10
ROLLOUT: 5
BATCHSIZE: 64
PRETRAIN: 1000


## Load the environment
<i> & print a bit of information contained in the wrapper class

In [4]:
env = E.Environment(args)
print("State size:", env.state_size)
print("Action size:", env.action_size)
print("Num Agents:", env.agent_count)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


State size: 33
Action size: 4
Num Agents: 20


<hr>

# Test code as it's developed

<hr>

## Take random actions in the environment below 
<i>
-to check that code is working<br>
-to get familiar with the environment

In [5]:
env.train = False
env.reset()

In [7]:
scores = np.zeros(env.agent_count)
states = env.states
for i in range(20):
    actions = np.random.randn(env.agent_count, env.action_size)
    actions = np.clip(actions, -1, 1)
    next_states, rewards, dones = env.step(actions)
    scores += rewards
    states = next_states
    if np.any(dones):
        break
    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

Total score (averaged over agents) this episode: 0.0


### Force-reload modules as they're updated
<i> This notebook was developed as the code is written in Atom, the below cell reloads the modules as they're needed.

In [ ]:
import agent
importlib.reload(agent)
#importlib.reload(E)
importlib.reload(models)
from agent import D4PG_Agent

In [11]:
d4pg_agent = D4PG_Agent(env.state_size, env.action_size, env.agent_count)
# print(d4pg_agent.__class__.__name__)
# print(d4pg_agent.memory)
# agent.initialize_memory(10, env)

In [12]:
env.reset()
states = env.states

### Test out Actor actions without training
<i> Test the <b>Actor</b> network

In [13]:
env.reset()
scores = np.zeros(env.agent_count)
states = env.states
for i in range(30):
    #actions = agent.actor(torch.from_numpy(states).float()).detach().numpy()
    #actions += agent.gauss_noise(actions.shape)
    #actions = np.clip(actions, -1, 1)
    actions = d4pg_agent.act(states)
    print("ACTIONS:", actions[1])
    next_states, rewards, dones = env.step(actions)
    scores += rewards
    states = next_states
    if np.any(dones):
        break
    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

ACTIONS: [ 0.39250484  0.10228997  0.31567538 -0.41675773]
ACTIONS: [ 0.22899312  0.2972572   0.08953724 -0.07314225]
ACTIONS: [ 0.5929218  -0.0166251  -0.22665542 -0.22307695]
ACTIONS: [ 0.4252136  -0.04706254 -0.29255268  0.5514186 ]
ACTIONS: [ 0.00659929  0.16880164 -0.15172873  0.34427068]
ACTIONS: [ 0.4874601  -0.05414694  0.18998487 -0.31815657]
ACTIONS: [0.5247522  0.28847387 0.7349592  0.18429683]
ACTIONS: [0.45776945 0.17300053 0.3757362  0.21077505]
ACTIONS: [ 0.54964024 -0.19631551  0.07580662  0.31457314]
ACTIONS: [ 0.32845622 -0.19134298 -0.48707473  0.02579423]
ACTIONS: [-0.03437502 -0.01179501  0.08437829  0.3584921 ]
ACTIONS: [-0.06189606 -0.21877146  0.16300105 -0.22402516]
ACTIONS: [ 0.06274641  0.61312693 -0.26764357  0.17627218]
ACTIONS: [-0.21768238  0.0877857  -0.01208796  0.05560011]
ACTIONS: [-0.4288527   0.2777754  -0.31315735 -0.13877532]
ACTIONS: [-0.64553875  0.30576026 -0.10319347 -0.07020348]
ACTIONS: [-0.22993162 -0.1994104  -0.1120761   0.17905626]
ACTIO

### Test out Critic scores without training
<i> Test the <b>Critic</b> network

In [39]:
env.reset()
scores = np.zeros(env.agent_count)
states = env.states
for i in range(10):
    actions = d4pg_agent.act(states) 
    next_states, rewards, dones = env.step(actions)
    scores += rewards
    q, probs = d4pg_agent.critic(next_states, torch.from_numpy(actions))
    #print(q.shape, q)
    #print(values.sample())
    states = next_states
    if np.any(dones):
        break
    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

Total score (averaged over agents) this episode: 0.0


In [ ]:
actions = np.random.randn(env.agent_count, env.action_size)
actions = np.clip(actions, -1, 1).astype(np.float32)
states = torch.from_numpy(env.states).float()

In [ ]:
q, probs, log_probs = critic(states, torch.from_numpy(actions))

In [ ]:
print(probs.shape)
print(log_probs.shape)
print(q)

In [49]:
rewards

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [54]:
r = torch.tensor(rewards).unsqueeze(-1)
r

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])

In [40]:
vmin = -10
vmax = 10
natoms = 51
gamma = .99
atoms = torch.linspace(vmin, vmax, natoms)
atoms

tensor([-10.0000,  -9.6000,  -9.2000,  -8.8000,  -8.4000,  -8.0000,  -7.6000,
         -7.2000,  -6.8000,  -6.4000,  -6.0000,  -5.6000,  -5.2000,  -4.8000,
         -4.4000,  -4.0000,  -3.6000,  -3.2000,  -2.8000,  -2.4000,  -2.0000,
         -1.6000,  -1.2000,  -0.8000,  -0.4000,   0.0000,   0.4000,   0.8000,
          1.2000,   1.6000,   2.0000,   2.4000,   2.8000,   3.2000,   3.6000,
          4.0000,   4.4000,   4.8000,   5.2000,   5.6000,   6.0000,   6.4000,
          6.8000,   7.2000,   7.6000,   8.0000,   8.4000,   8.8000,   9.2000,
          9.6000,  10.0000])

In [53]:
atoms.view(1,-1)

tensor([[-10.0000,  -9.6000,  -9.2000,  -8.8000,  -8.4000,  -8.0000,  -7.6000,
          -7.2000,  -6.8000,  -6.4000,  -6.0000,  -5.6000,  -5.2000,  -4.8000,
          -4.4000,  -4.0000,  -3.6000,  -3.2000,  -2.8000,  -2.4000,  -2.0000,
          -1.6000,  -1.2000,  -0.8000,  -0.4000,   0.0000,   0.4000,   0.8000,
           1.2000,   1.6000,   2.0000,   2.4000,   2.8000,   3.2000,   3.6000,
           4.0000,   4.4000,   4.8000,   5.2000,   5.6000,   6.0000,   6.4000,
           6.8000,   7.2000,   7.6000,   8.0000,   8.4000,   8.8000,   9.2000,
           9.6000,  10.0000]])

In [42]:
atoms_delta = (vmax - vmin) / (natoms -1)
atoms_delta

0.4

In [43]:
probs = probs.detach()
probs

tensor([[0.0196, 0.0197, 0.0196,  ..., 0.0196, 0.0196, 0.0196],
        [0.0196, 0.0197, 0.0196,  ..., 0.0196, 0.0196, 0.0196],
        [0.0196, 0.0197, 0.0196,  ..., 0.0196, 0.0196, 0.0196],
        ...,
        [0.0196, 0.0197, 0.0196,  ..., 0.0196, 0.0196, 0.0196],
        [0.0196, 0.0197, 0.0196,  ..., 0.0196, 0.0196, 0.0196],
        [0.0196, 0.0197, 0.0196,  ..., 0.0196, 0.0196, 0.0196]])

In [44]:
q_next = (probs * atoms).sum()
q_next

tensor(-0.0289)

In [56]:
print(r.shape)
print(atoms.view(1,-1).shape)

torch.Size([20, 1])

atoms.view(1,-1) becomes shape [1, num_atoms]

r is unsqueezed in the last (-1) dimension, so it's shape [20,1]

the result is a tensor that holds an offset (projected) version of the atoms for each reward instance

In [60]:
projected_atoms = r + gamma * atoms.view(1,-1)
projected_atoms.clamp_(vmin, vmax)

tensor([[-9.9000, -9.5040, -9.1080,  ...,  9.1080,  9.5040,  9.9000],
        [-9.9000, -9.5040, -9.1080,  ...,  9.1080,  9.5040,  9.9000],
        [-9.9000, -9.5040, -9.1080,  ...,  9.1080,  9.5040,  9.9000],
        ...,
        [-9.9000, -9.5040, -9.1080,  ...,  9.1080,  9.5040,  9.9000],
        [-9.9000, -9.5040, -9.1080,  ...,  9.1080,  9.5040,  9.9000],
        [-9.9000, -9.5040, -9.1080,  ...,  9.1080,  9.5040,  9.9000]])

In [62]:
b = (projected_atoms - vmin) / atoms_delta
b

tensor([[ 0.2500,  1.2400,  2.2300,  ..., 47.7700, 48.7600, 49.7500],
        [ 0.2500,  1.2400,  2.2300,  ..., 47.7700, 48.7600, 49.7500],
        [ 0.2500,  1.2400,  2.2300,  ..., 47.7700, 48.7600, 49.7500],
        ...,
        [ 0.2500,  1.2400,  2.2300,  ..., 47.7700, 48.7600, 49.7500],
        [ 0.2500,  1.2400,  2.2300,  ..., 47.7700, 48.7600, 49.7500],
        [ 0.2500,  1.2400,  2.2300,  ..., 47.7700, 48.7600, 49.7500]])

In [81]:
l = b.floor().long()
u = b.ceil().long()
print(l[1])
print(u[2])

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])
tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])


In [67]:
dml = (u + (l == u).float() - b) * probs
dml[0]

tensor([0.0147, 0.0149, 0.0151, 0.0153, 0.0155, 0.0157, 0.0159, 0.0160, 0.0163,
        0.0165, 0.0167, 0.0169, 0.0171, 0.0173, 0.0174, 0.0177, 0.0178, 0.0180,
        0.0182, 0.0184, 0.0187, 0.0188, 0.0190, 0.0192, 0.0194, 0.0196, 0.0002,
        0.0004, 0.0006, 0.0008, 0.0010, 0.0012, 0.0014, 0.0016, 0.0018, 0.0020,
        0.0022, 0.0024, 0.0026, 0.0027, 0.0029, 0.0031, 0.0033, 0.0035, 0.0037,
        0.0039, 0.0041, 0.0043, 0.0045, 0.0047, 0.0049])

In [88]:
dml = l.float() + (probs * (u.float()-b))
dml

tensor([[1.4678e-02, 1.0149e+00, 2.0151e+00,  ..., 4.7005e+01, 4.8005e+01,
         4.9005e+01],
        [1.4680e-02, 1.0149e+00, 2.0151e+00,  ..., 4.7005e+01, 4.8005e+01,
         4.9005e+01],
        [1.4678e-02, 1.0149e+00, 2.0151e+00,  ..., 4.7005e+01, 4.8005e+01,
         4.9005e+01],
        ...,
        [1.4678e-02, 1.0149e+00, 2.0151e+00,  ..., 4.7005e+01, 4.8005e+01,
         4.9005e+01],
        [1.4679e-02, 1.0149e+00, 2.0151e+00,  ..., 4.7005e+01, 4.8005e+01,
         4.9005e+01],
        [1.4679e-02, 1.0149e+00, 2.0151e+00,  ..., 4.7005e+01, 4.8005e+01,
         4.9005e+01]])

In [90]:
dmu = (b - l.float()) * probs
dmu[0]

tensor([0.0049, 0.0047, 0.0045, 0.0043, 0.0041, 0.0039, 0.0037, 0.0035, 0.0033,
        0.0031, 0.0029, 0.0027, 0.0025, 0.0024, 0.0022, 0.0020, 0.0018, 0.0016,
        0.0014, 0.0012, 0.0010, 0.0008, 0.0006, 0.0004, 0.0002, 0.0000, 0.0194,
        0.0193, 0.0190, 0.0188, 0.0186, 0.0184, 0.0182, 0.0180, 0.0179, 0.0177,
        0.0174, 0.0173, 0.0171, 0.0168, 0.0166, 0.0164, 0.0163, 0.0160, 0.0159,
        0.0157, 0.0155, 0.0153, 0.0151, 0.0149, 0.0147])

In [76]:
target_prob = torch.tensor(np.zeros(probs.size()))
target_prob.shape

torch.Size([20, 51])

In [91]:
for i in range(target_prob.size(0)):
    target_prob[i].index_add_(0, l[i].long(), dml[i].double())
    target_prob[i].index_add_(0, u[i].long(), dmu[i].double())
target_prob[0]

tensor([5.8712e-02, 1.0744e+00, 2.0746e+00, 3.0748e+00, 4.0748e+00, 5.0752e+00,
        6.0754e+00, 7.0753e+00, 8.0757e+00, 9.0760e+00, 1.0076e+01, 1.1076e+01,
        1.2076e+01, 1.3077e+01, 1.4077e+01, 1.5077e+01, 1.6077e+01, 1.7077e+01,
        1.8078e+01, 1.9078e+01, 2.0078e+01, 2.1078e+01, 2.2079e+01, 2.3079e+01,
        2.4079e+01, 5.0060e+01, 2.6060e+01, 2.7060e+01, 2.8060e+01, 2.9060e+01,
        3.0061e+01, 3.1061e+01, 3.2061e+01, 3.3061e+01, 3.4062e+01, 3.5062e+01,
        3.6062e+01, 3.7062e+01, 3.8062e+01, 3.9062e+01, 4.0062e+01, 4.1063e+01,
        4.2063e+01, 4.3063e+01, 4.4063e+01, 4.5064e+01, 4.6064e+01, 4.7064e+01,
        4.8064e+01, 4.9064e+01, 4.4159e-02], dtype=torch.float64)

In [ ]:
env.close()